(c) 2016 Enplus Advisors, Inc.

In [1]:
import numpy as np
import pandas as pd

# Merge, Join, and Pivot

## Revisit our familiar DataFrame

In [2]:
symbols = ['AAPL', 'MSFT', 'IBM', 'YHOO']
_df = {
    'ticker': symbols,
    'date': ['2015-12-30', '2015-12-30', '2015-12-30', '2015-12-30'],
    'open': [426.23, 42.3, 101.65, 35.53]
}
df = pd.DataFrame(_df).set_index('ticker')
df

,date,open
ticker,,
AAPL,2015-12-30,426.23
MSFT,2015-12-30,42.30
IBM,2015-12-30,101.65
YHOO,2015-12-30,35.53


## Index Join

In [3]:
symbols2 = symbols + ['MANT']
s1 = pd.Series(range(1, 6), symbols2, name='close')

In [4]:
df.join(s1)

,date,open,close
ticker,,,
AAPL,2015-12-30,426.23,1
MSFT,2015-12-30,42.30,2
IBM,2015-12-30,101.65,3
YHOO,2015-12-30,35.53,4


In [5]:
df.join(s1, how='outer')

,date,open,close
AAPL,2015-12-30,426.23,1
IBM,2015-12-30,101.65,3
MANT,NaN,NaN,5
MSFT,2015-12-30,42.30,2
YHOO,2015-12-30,35.53,4


## Merge Join

In [6]:
df_close = pd.DataFrame(s1)
df_close.index.name = 'ticker'
df_close.reset_index(inplace=True)

In [7]:
pd.merge(df, df_close, left_index=True, right_on='ticker')

,date,open,ticker,close
0,2015-12-30,426.23,AAPL,1
1,2015-12-30,42.30,MSFT,2
2,2015-12-30,101.65,IBM,3
3,2015-12-30,35.53,YHOO,4


## Pivot

In [8]:
df

,date,open
ticker,,
AAPL,2015-12-30,426.23
MSFT,2015-12-30,42.30
IBM,2015-12-30,101.65
YHOO,2015-12-30,35.53


In [9]:
df['close'] = np.random.randn(4)
df1 = df.reset_index().set_index(['ticker', 'date'])
df1

,,open,close
ticker,date,,
AAPL,2015-12-30,426.23,-0.501986
MSFT,2015-12-30,42.30,-0.534366
IBM,2015-12-30,101.65,0.493307
YHOO,2015-12-30,35.53,1.857758


In [10]:
df_long = df1.stack().reset_index().rename(columns={'level_2': 'variable', 0: 'value'})
df_long

,ticker,date,variable,value
0,AAPL,2015-12-30,open,426.230000
1,AAPL,2015-12-30,close,-0.501986
2,MSFT,2015-12-30,open,42.300000
3,MSFT,2015-12-30,close,-0.534366
4,IBM,2015-12-30,open,101.650000
5,IBM,2015-12-30,close,0.493307
6,YHOO,2015-12-30,open,35.530000
7,YHOO,2015-12-30,close,1.857758


In [11]:
res = []
for _, df, in df_long.groupby('ticker'):
    tmp = df[df.value == df.value.max()]
    res.append(tmp)
pd.concat(res, ignore_index=True)

,ticker,date,variable,value
0,AAPL,2015-12-30,open,426.23
1,IBM,2015-12-30,open,101.65
2,MSFT,2015-12-30,open,42.30
3,YHOO,2015-12-30,open,35.53


In [12]:
def keep_max(df):
    return df[df.value == df.value.max()]
df_long.groupby('ticker').apply(keep_max)

,,ticker,date,variable,value
ticker,,,,,
AAPL,0,AAPL,2015-12-30,open,426.23
IBM,4,IBM,2015-12-30,open,101.65
MSFT,2,MSFT,2015-12-30,open,42.30
YHOO,6,YHOO,2015-12-30,open,35.53


In [13]:
df_long.drop_duplicates('ticker', keep='first')

,ticker,date,variable,value
0,AAPL,2015-12-30,open,426.23
2,MSFT,2015-12-30,open,42.30
4,IBM,2015-12-30,open,101.65
6,YHOO,2015-12-30,open,35.53
